

 # Connect to our Worskpace

In [ ]:
#Let's connect to our workspace:
from azureml.core import Workspace
ws = Workspace.from_config(path=".azureml/ws_config.json")
ws

# Reference our registered model:

In [ ]:
#Let's reference our registered model:
from azureml.core.model import Model
print(Model.list(workspace = ws))
print("______________________________")
print(Model.get_model_path('sklearn_bh_lr'))
print("______________________________")
model_path = Model.get_model_path('sklearn_bh_lr')
model = Model(ws,'sklearn_bh_lr')
print(model)
print("______________________________")
print(type(model))


In [ ]:
!ls -al azureml-models/sklearn_bh_lr/

# Build an entry script

In [ ]:
import json
import pickle
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from azureml.core.model import Model

sample = json.dumps({"data": [7.534]})
print(sample)
print(type(sample))
print(json.loads(sample)['data'])
my_array = np.array(json.loads(sample)['data'])
print(my_array.shape)
my2d_array = my_array.reshape(-1,1)
print(my2d_array)
print(my2d_array.shape)
##test_samples = bytes(test_samples, encoding='utf8')

#print(Model.list(ws))
model_path = Model.get_model_path(model_name = 'sklearn_bh_lr')
print(model_path)

local_model = joblib.load(model_path)
print("local_model: ", local_model.predict(my2d_array))


def run(data):
    data = np.array(json.loads(data)['data'])
    result = local_model.predict(data.reshape(-1,1))
    
    return result.tolist()
    
print("local_model_through_run_function: ", run(sample))  


In [ ]:
%%writefile score.py
# Now we build an entry script that uses our model reference:

import json
import pickle
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from azureml.core.model import Model



def init():
    global model
    model_path = Model.get_model_path(model_name = 'sklearn_bh_lr')
    model = joblib.load(model_path)
    
    
    
def run(data):
    data = np.array(json.loads(data)['data'])
    result = model.predict(data.reshape(-1,1))
    
    return result.tolist()
    


# Build Dependency list and save it as a yaml file.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.set_python_version("3.6.6")
dependencies.add_pip_package("scikit-learn==0.20.3")
dependencies.add_pip_package("numpy==1.16.2")
dependencies.add_pip_package("pandas==0.23.4")
dependencies.add_pip_package("matplotlib==3.0.0")

dependencies.save_to_file(".","myenv.yml")

In [ ]:
!ls

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage

# Define deplolyment configuration and container image

In [ ]:
# Define the configuration of compute: ACI with 1 cpu core and 1 gb of memory.
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aci_config



In [ ]:
# Build a container image
image_config = ContainerImage.image_configuration(execution_script = "score.py", 
                                   runtime = "python", 
                                   conda_file = "myenv.yml")
image_config


In [ ]:
# Deploy the web service as an image containing the registered model.

service = Webservice.deploy_from_model(name = "housing-predictor",
                                        deployment_config = aci_config,
                                        models = [model],
                                        image_config = image_config,
                                        workspace = ws)

# The service deployment can take several minutes: wait for completion.
service.wait_for_deployment(show_output = True)

In [ ]:
#service = Webservice(ws, 'housing-predictor')

In [ ]:
#print(service.get_logs())

In [ ]:
result = service.run(sample)
#json.loads(sample)['data']
print("return_from_aci_deployment: ", result)
#my_model = joblib.load(model_path)
#data = np.array([1,3,5,7])
#data
#data.size



In [ ]:
service.delete()